In [1]:
import _pickle
fd=open('model.pkl','rb')
clf=_pickle.load(fd)

In [6]:
import pandas as pd
import numpy as np
def test(clf):
    origTestDf = pd.read_csv("test.csv")
    users = origTestDf.user
    events = origTestDf.event
    testDf = pd.read_csv("newdata/test.csv")
    testDf.fillna(0,inplace=True)
    fout = open("newdata/result.csv", 'w')
    fout.write(",".join(["user", "event", "outcome", "dist"]) + "\n")
    nrows = len(testDf)
    Xp = np.matrix(testDf)
    yp = np.zeros((nrows, 2))
    for i in range(0, nrows):
        xp = Xp[i, :]
        yp[i, 0] = clf.predict(xp)
        yp[i, 1] = clf.decision_function(xp)
        fout.write(",".join(map(lambda x: str(x), [users[i], events[i], yp[i, 0], yp[i, 1]])) + "\n")
    fout.close()
test(clf)

In [7]:
from __future__ import division

import pandas as pd

def byDist(x, y):
    return int(y[1] - x[1])

def main():
    # output file
    fout = open("newdata/finalrecommendation.csv", 'w')
    fout.write(",".join(["User", "Events"]) + "\n")
    resultDf = pd.read_csv("newdata/result.csv")
    # group remaining user/events
    grouped = resultDf.groupby("user")
    for name, group in grouped:
        user = str(name)
        tuples = zip(list(group.event), list(group.dist), list(group.outcome))
        #   tuples = filter(lambda x: x[2]==1, tuples)
        tuples = sorted(tuples, key=lambda x: x[1],reverse=True)
        events = "\"" + str(list(map(lambda x: x[0], tuples))) + "\""
        fout.write(",".join([user, events]) + "\n")
    fout.close()
main()